In [1]:
!pip install "uvicorn[standard]"
!pip install fastapi

     -------------------------------------- 58.3/58.3 kB 139.8 kB/s eta 0:00:00
     -------------------------------------- 58.3/58.3 kB 615.6 kB/s eta 0:00:00
     ------------------------------------ 143.0/143.0 kB 387.0 kB/s eta 0:00:00
     -------------------------------------- 270.9/270.9 kB 1.2 MB/s eta 0:00:00
     -------------------------------------- 124.6/124.6 kB 2.4 MB/s eta 0:00:00
     -------------------------------------- 57.0/57.0 kB 129.8 kB/s eta 0:00:00
     ---------------------------------------- 2.1/2.1 MB 1.9 MB/s eta 0:00:00
     ---------------------------------------- 66.9/66.9 kB 1.8 MB/s eta 0:00:00


In [7]:
import h2o
from h2o.automl import H2OAutoML
from fastapi import FastAPI
from pydantic import BaseModel

h2o.init()
saved_model = h2o.load_model("StackedEnsemble_BestOfFamily_1_AutoML_2_20230505_53230")
app = FastAPI()

class InputData(BaseModel):
    vol_moving_avg: float
    adj_close_rolling_med: float
    symbol: str
    etf: bool
    rsi: float
    tema: float

@app.post("/predict")
def predict_trading_volume(input_data: InputData):
    # Extract input values from the request body
    vol_moving_avg = input_data.vol_moving_avg
    adj_close_rolling_med = input_data.adj_close_rolling_med
    symbol = input_data.symbol
    etf = input_data.etf
    rsi = input_data.rsi
    tema = input_data.tema
    
    frame = h2o.H2OFrame({"Symbol":symbol, "etf":etf, "rsi":rsi, "tema":tema, "vol_moving_avg":vol_moving_avg, "adj_close_rolling_med":adj_close_rolling_med})
    result = saved_model.predict(frame)
    print(result.as_data_frame()[1][0])
    # Return the predicted trading volume as a response
    return {"trading_volume": result.as_data_frame()[1][0]}

In [15]:
import requests
url = "http://127.0.0.1:8000/predict"

payload = {
    "vol_moving_avg": 2212460,
    "adj_close_rolling_med": 36.5099,
    "symbol": "A",
    "etf": False,
    "rsi": 58.1535,
    "tema": 39.4062
}

headers = {
    "Content-Type": "application/json",
    "accept": "application/json"
}

response = requests.post(url, json=payload, headers=headers)

print(response.json())

{'trading_volume': '1378170.4745345903'}
